In [1]:
#Keystroke_Manhattan.py
 
from scipy.spatial.distance import cityblock
import numpy as np
np.set_printoptions(suppress = True)
import pandas as pd
from sklearn import metrics

#function to evaluate equal error rate (EER) for the detector
def evaluateEER(user_scores, imposter_scores):
    labels = [0]*len(user_scores) + [1]*len(imposter_scores)
    fpr, tpr, thresholds = metrics.roc_curve(labels, user_scores + imposter_scores)
    missrates = 1 - tpr
    farates = fpr
    dists = missrates - farates
    idx1 = np.argmin(dists[dists >= 0])
    idx2 = np.argmax(dists[dists < 0])
    x = [missrates[idx1], farates[idx1]]
    y = [missrates[idx2], farates[idx2]]
    a = ( x[0] - x[1] ) / ( (y[1] - y[0]) - (x[1] - x[0]) )
    eer = x[0] + a * ( y[0] - x[0] )
    
    plt.figure(figsize = (10,5))
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr, tpr, color = 'red',label='AUC = {:.3f}, EER = {:.3f} One Class SVM'.format(auc(fpr, tpr), eer))
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.title('ROC curve for ')
    plt.legend(loc='best')
    plt.show()
    print("One Class distance shows error rates ", eer)
    
    return eer
 
class ManhattanDetector:
  
    def __init__(self, subjects):
        self.user_scores = []
        self.imposter_scores = []
        self.mean_vector = []
        self.subjects = subjects
  
    #calculates mean for each user from training samples
    def training(self):
        self.mean_vector = self.train.mean().values
        #print(self.mean_vector)
    
    #calculating manhattan distance
    def testing(self):
        for i in range(self.test_genuine.shape[0]):
            cur_score = cityblock(self.test_genuine.iloc[i].values, self.mean_vector)
            self.user_scores.append(cur_score)
  
        for i in range(self.test_imposter.shape[0]):
            cur_score = cityblock(self.test_imposter.iloc[i].values, self.mean_vector)
            self.imposter_scores.append(cur_score)
  
    def evaluate(self):
        eers = []
  
        for subject in subjects:        
            genuine_user_data = data.loc[data.subject == subject, "H.period":"H.Return"]
            imposter_data = data.loc[data.subject != subject, :]
            
            #training dataset- consisting of first 300 samples
            self.train = genuine_user_data[:300]
            
            #testing dataset
            #genuine samples of remaining 100 samples
            self.test_genuine = genuine_user_data[300:]
            #imposter dataset- for a user 5 samples of remaining users are taken
            self.test_imposter = imposter_data.groupby("subject"). head(5).loc[:, "H.period":"H.Return"]
            
            self.training()
            self.testing()
            eers.append(evaluateEER(self.user_scores, self.imposter_scores))
        return np.mean(eers)
 
#
#loading of dataset
data = pd.read_csv("C:/Users/Sanskriti/Downloads/DSL-StrongPasswordData.csv")
subjects = data["subject"].unique()
print ("average EER for Manhattan detector: ")
print(ManhattanDetector(subjects).evaluate())

average EER for Manhattan detector: 
0.158807700030822
